In [1]:
#import packages
import requests
import json
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import datetime


In [2]:
path = "Park Unit Scraping Information.csv"
parks = pd.read_csv(path, encoding='latin-1')
parks.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Abraham Lincoln Birthplace,Abraham Lincoln Birthplace National Historical...,National Historical Park,KY,SER,ABLI,1.0
1,Acadia,Acadia National Park,National Park,ME,NER,ACAD,1.0
2,Adams,Adams National Historical Park,National Historical Park,MA,NER,ADAM,1.0
3,African Burial Ground,African Burial Ground National Monument,National Monument,NY,NER,AFBG,1.0
4,Agate Fossil Beds,Agate Fossil Beds National Monument,National Monument,NE,MWR,AGFO,1.0


In [27]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [55]:
id_list = []
url_list = []
title_list = []
parkCode_list = []
description_list = []
category_list = []
relatedRoadEvents_list = []
lastIndexedDate_list = []

for x in df['data']:
    #print(x)
    id_list.append(x['id'])
    url_list.append(x['url']) 
    title_list.append(x['title']) 
    parkCode_list.append(x['parkCode'].upper()) 
    description_list.append(x['description'])
    category_list.append(x['category']) 
    relatedRoadEvents_list.append(x['relatedRoadEvents'])
    lastIndexedDate_list.append(x['lastIndexedDate'])


In [56]:
alerts_df = pd.DataFrame({
    'id': id_list, 
    'url': url_list, 
    'title': title_list,
    'park alpha': parkCode_list,
    'description': description_list,
    'category': category_list,
    'relatedRoadEvents':relatedRoadEvents_list,
    'LastIndexedDate':lastIndexedDate_list
})

In [57]:
alerts_df.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate
0,EB67B609-8B75-49BB-9E66-8B1B657E8343,,Kelabker Road and National Trails Hwy CLOSED b...,JOTR,This is outside of Joshua Tree National Park a...,Information,[],2024-02-28 14:31:49.0
1,528CA0BF-D5FC-4034-B1F8-CC551E2C3BF5,https://www.nps.gov/chis/planyourvisit/creek-c...,"Wet Creek Crossing at Canada Del Puerto, Priso...",CHIS,"During winter and spring, visitors need to be ...",Caution,[],2024-02-28 14:03:45.0
2,E6BCCA5D-24E9-49C2-9F4E-7C16D3BCA3E5,https://forecast.weather.gov/MapClick.php?lat=...,Changing conditions - Hazardous Weather,LAVO,"Blizzard Warning in effect from February 29, 0...",Caution,[],2024-02-28 13:30:39.0
3,9AC0D028-37B3-442B-8C14-36AF11EDA4AC,https://www.nps.gov/bibe/advice-for-busy-times...,Big Bend Spring = Full Campgrounds,BIBE,March is the busiest time of the year in Big B...,Information,[],2024-02-28 13:01:20.0
4,F208C6B1-54EB-48B9-A0CD-C149BE25DF53,https://www.nps.gov/redw/planyourvisit/conditi...,Winter Storm Related Closures,REDW,A winter storm starting on Wednesday evening (...,Park Closure,"[{'title': 'Coastal Drive Closed', 'id': 'F208...",2024-02-28 12:35:02.0


In [58]:
alerts_df.category.value_counts()

Information     357
Park Closure    208
Caution         125
Danger           16
                  2
Name: category, dtype: int64

In [69]:
counter = 0
road_closed_list = []

for x in range(0,len(alerts_df)):
    road_closed = 0
    if "lane closure" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if "road closure" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if "road closed" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if "road is closed" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if ("road" in str(alerts_df['description'][x]).lower())&("closed" in str(alerts_df['description'][x]).lower()):
        road_closed=1
    road_closed_list.append(road_closed)

    if road_closed == 1:
        print(alerts_df['park alpha'][x])
        print(alerts_df['description'][x])
        counter +=1

alerts_df['road_closed_flag'] = road_closed_list
    


SUCR
The Sunset Crater Volcano Visitor Center will be closed on Thursday 2/29/24. All roads and trails remain open. Thank you for helping us protect park resources as you visit.
CRLA
The road is closed while plow operators work to clear snow and prepare for the next big storm arriving late Wednesday afternoon. It is not possible to access a lake view by vehicle at this time. Expect snowy and icy roads. All Oregon Chain Laws apply.
CANY
The unpaved Elephant Hill Access Road and trailhead parking lot will close temporarily for necessary road maintenance on March 21, 2024. The road and parking lot will be closed to all uses until the work is complete, which should be later that same day.
LAMR
Due to ongoing road work Blue West is closed until further notice.
SCBL
Due to snow and ice on the roadway surface, the Scotts Bluff Summit Road is closed until further notice.
YOSE
Tioga Road (continuation of Highway 120 through the park) and Glacier Point Road are closed for the season due to snow.

In [70]:
sum(alerts_df['road_closed_flag'])

58

In [61]:
alerts_df2 = alerts_df.merge(parks,left_on='park alpha', right_on="Alpha", how = "left")

In [75]:
alerts_df2['relatedRoadEvents'].value_counts()

[]                                                                                                                                                                                                                            685
[{'title': 'Coastal Drive Closed', 'id': 'F208C6B1-54EB-48B9-A0CD-C149BE25DF53', 'type': 'roadevent', 'url': 'https://www.nps.gov/redw/planyourvisit/conditions.htm'}]                                                          1
[{'title': 'Gachado to Dos Lomitas Closure', 'id': '26FB2023-E141-4E46-A913-0E0996048633', 'type': 'roadevent', 'url': 'https://www.nps.gov/orpi/planyourvisit/conditions.htm'}]                                                1
[{'title': 'Yarborough Pass Closed', 'id': '566BD223-0759-4322-86E5-58FD229E9F15', 'type': 'roadevent', 'url': 'https://www.nps.gov/pais/planyourvisit/conditions.htm'}]                                                        1
[{'title': 'Tanners Ridge Fire Road', 'id': 'C509E46A-FE91-4AE4-9D13-9ECFEFC2F89D', 'type': 'roa

In [87]:
alerts_df2.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,road_closed_flag,Park Name,Type of Unit,State,Reg,Alpha,Unit
0,EB67B609-8B75-49BB-9E66-8B1B657E8343,,Kelabker Road and National Trails Hwy CLOSED b...,JOTR,This is outside of Joshua Tree National Park a...,Information,[],2024-02-28 14:31:49.0,0,Joshua Tree,National Park,CA,PWR,JOTR,1.0
1,528CA0BF-D5FC-4034-B1F8-CC551E2C3BF5,https://www.nps.gov/chis/planyourvisit/creek-c...,"Wet Creek Crossing at Canada Del Puerto, Priso...",CHIS,"During winter and spring, visitors need to be ...",Caution,[],2024-02-28 14:03:45.0,0,Channel Islands,National Park,CA,PWR,CHIS,1.0
2,E6BCCA5D-24E9-49C2-9F4E-7C16D3BCA3E5,https://forecast.weather.gov/MapClick.php?lat=...,Changing conditions - Hazardous Weather,LAVO,"Blizzard Warning in effect from February 29, 0...",Caution,[],2024-02-28 13:30:39.0,1,Lassen Volcanic,National Park,CA,PWR,LAVO,1.0
3,9AC0D028-37B3-442B-8C14-36AF11EDA4AC,https://www.nps.gov/bibe/advice-for-busy-times...,Big Bend Spring = Full Campgrounds,BIBE,March is the busiest time of the year in Big B...,Information,[],2024-02-28 13:01:20.0,0,Big Bend,National Park,TX,IMR,BIBE,1.0
4,F208C6B1-54EB-48B9-A0CD-C149BE25DF53,https://www.nps.gov/redw/planyourvisit/conditi...,Winter Storm Related Closures,REDW,A winter storm starting on Wednesday evening (...,Park Closure,"[{'title': 'Coastal Drive Closed', 'id': 'F208...",2024-02-28 12:35:02.0,0,Redwood,National and State Parks,CA,PWR,REDW,1.0


In [90]:
alerts_df2.loc[alerts_df2['relatedRoadEvents'].str.contains("nps.gov")==True]

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,road_closed_flag,Park Name,Type of Unit,State,Reg,Alpha,Unit


In [71]:
alerts_df_closures = alerts_df2.loc[alerts_df2['road_closed_flag']==1]
alerts_df_closures.Reg.value_counts()

IMR    30
PWR    16
SER    13
NCR     8
MWR     7
NER     2
AKR     2
PW      2
Name: Reg, dtype: int64

In [66]:
alerts_df2.to_csv("alerts_api.csv", index=False)